In [1]:
import json
from math import sqrt
from queue import PriorityQueue

In [2]:
def ucs_without_budget(graph, dist, src, dest):

    # create a priority queue
    queue = PriorityQueue()

    # push the starting index and path
    queue.put([0, [src]])

    # dictionary to keep track of visited node
    visited = {}

    # while the queue is not empty
    while queue:

        # pop the element with the highest priority
        e = queue.get()

        # get the current distance
        cur_dist = e[0]

        # get the current path
        cur_path = e[1]

        # set the current node to the last node in the path
        cur_node = cur_path[-1]

        # check if current node is the destination node
        if cur_node == dest:
            # return the path and the total distance from source to destination node
            return cur_path, cur_dist

        # check for the non visited nodes
        if cur_node not in visited:
            for neighbour in graph[cur_node]:

                # clone current path to a new path to 
                # prevent appending to the current path
                newPath = cur_path[:]

                # append the adjacent node to the new path
                newPath.append(neighbour)

                # calculate the new distance
                newDist = cur_dist + dist[f"{cur_node},{neighbour}"]

                # push adjacent node with it's distance and path into priority queue
                queue.put([newDist, newPath])

        # mark current ndoe as visited
        visited[cur_node] = 1

In [3]:
def ucs_with_budget(graph, dist, cost, src, dest, max_energy_cost):

    # create a priority queue
    queue = PriorityQueue()

    # push the starting index and path
    queue.put([0, 0, [src]])

    # dictionary to keep track of visited node
    visited = {}

    # while the queue is not empty
    while queue:

        # pop the element with the highest priority
        e = queue.get()

        # get the current distance
        cur_dist = e[0]

        # get the current energy cost
        cur_cost = e[1]

        # get the current path
        cur_path = e[2]

        # set the current node to the last node in the path
        cur_node = cur_path[-1]

        # check if current node is the destination node
        if cur_node == dest:
            # return the path and the total distance from source to destination node
            return cur_path, cur_dist, cur_cost

        # check for the non visited nodes
        if cur_node not in visited:
            for neighbour in graph[cur_node]:

                # clone current path to a new path to
                # prevent appending to the current path
                newPath = cur_path[:]

                # append the adjacent node to the new path
                newPath.append(neighbour)

                # calculcate the new cost
                newCost = cur_cost + cost[f"{cur_node},{neighbour}"]

                # calculate the new distance
                newDist = cur_dist + dist[f"{cur_node},{neighbour}"]

                # check if new cost is less than the max energy cost
                if newCost <= max_energy_cost:
                    # push adjacent node with it's distance and path into priority queue
                    queue.put([newDist, newCost, newPath])

        # mark current ndoe as visited
        visited[cur_node] = 1

In [4]:
def heuristic(dest, neighbour, coord):
    (x1, y1) = coord[dest]
    (x2, y2) = coord[neighbour]
    # find the straight line distance between two ndoes
    return sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))
    
def a_star_search(graph, dist, cost, coord, src, dest, max_energy_cost):

    # create a priority queue
    queue = PriorityQueue()

    # push the starting index and path
    queue.put([0, 0, 0, [src]])

    # dictionary to keep track of visited node
    visited = {}

    # while the queue is not empty
    while queue:

        # pop the element with the highest priority
        e = queue.get()
        
        # get the current distance
        cur_dist = e[1]

        # get the current energy cost
        cur_cost = e[2]

        # get the current path
        cur_path = e[3]

        # set the current node to the last node in the path
        cur_node = cur_path[-1]

        # check if current node is the destination node
        if cur_node == dest:
            # return the path and the total distance from source to destination node
            return cur_path, cur_dist, cur_cost

        # check for the non visited nodes
        for neighbour in graph[cur_node]:

            # clone current path to a new path to
            # prevent appending to the current path
            newPath = cur_path[:]

            # append the adjacent node to the new path
            newPath.append(neighbour)

            # calculcate the new cost
            newCost = cur_cost + cost[f"{cur_node},{neighbour}"]

            # calculate the new distance
            newDist = cur_dist + dist[f"{cur_node},{neighbour}"]

            if neighbour not in visited or newDist < cur_dist:
                # check if new cost is less than the max energy cost
                if newCost <= max_energy_cost:
                    # calculate the new priority
                    priority = newDist + heuristic(dest, neighbour, coord)
                    
                    # push adjacent node with it's priority, distance and path into priority queue
                    queue.put([priority, newDist, newCost, newPath])

        # mark current ndoe as visited
        visited[cur_node] = 1


In [5]:
if __name__ == '__main__':

    # load graph from JSON
    g = open("G.json")
    graph = json.load(g)

    # load distance from JSON
    d = open("Dist.json")
    dist = json.load(d)

    # load cost from JSON
    c = open("Cost.json")
    cost = json.load(c)

    # load coord from JSON
    cd = open("coord.json")
    coord = json.load(cd)

    # set source node
    src = '1'

    # set destination node
    dest = '50'

    # set max energy cost
    max_energy_cost = 287932

    print("Task 1 - Uniform Cost Search to solve a relaxed version of the NYC instance where we do not have the energy constraint")

    # find shortest distance from source to destination node without energy budget using Uniform Cost Search
    path, shortest_dist = ucs_without_budget(graph, dist, src, dest)

    # print the shortest path
    print("Shortest path: ", end="")
    print(*path, sep=" -> ", end=".\n")

    # print the shortest distance
    print(f"Shortest Distance: {shortest_dist}.\n")

    print("Task 2 - Uniform Cost Search to solve the NYC instance")

    # find shortest distance from source to destination node with energy budget using Uniform Cost Search
    path, shortest_dist, total_energy_cost = ucs_with_budget(graph, dist, cost, src, dest, max_energy_cost)

    # print the shortest path
    print("Shortest path: ", end="")
    print(*path, sep=" -> ", end=".\n")

    # print the shortest distance
    print(f"Shortest Distance: {shortest_dist}.")

    # print the total energy cost
    print(f"Total energy cost: {total_energy_cost}.\n")

    print("Task 3 - A* search algorithm to solve the NYC instance")

    # find shortest distance from source to destination node with energy budget using A* Search
    path, shortest_dist, total_energy_cost = a_star_search(graph, dist, cost, coord, src, dest, max_energy_cost)

    # print the shortest path
    print("Shortest path: ", end="")
    print(*path, sep=" -> ", end=".\n")

    # print the shortest distance
    print(f"Shortest Distance: {shortest_dist}.")

    # print the total energy cost
    print(f"Total energy cost: {total_energy_cost}.\n")

Task 1 - Uniform Cost Search to solve a relaxed version of the NYC instance where we do not have the energy constraint
Shortest path: 1 -> 1363 -> 1358 -> 1357 -> 1356 -> 1276 -> 1273 -> 1277 -> 1269 -> 1267 -> 1268 -> 1284 -> 1283 -> 1282 -> 1255 -> 1253 -> 1260 -> 1259 -> 1249 -> 1246 -> 963 -> 964 -> 962 -> 1002 -> 952 -> 1000 -> 998 -> 994 -> 995 -> 996 -> 987 -> 986 -> 979 -> 980 -> 969 -> 977 -> 989 -> 990 -> 991 -> 2369 -> 2366 -> 2340 -> 2338 -> 2339 -> 2333 -> 2334 -> 2329 -> 2029 -> 2027 -> 2019 -> 2022 -> 2000 -> 1996 -> 1997 -> 1993 -> 1992 -> 1989 -> 1984 -> 2001 -> 1900 -> 1875 -> 1874 -> 1965 -> 1963 -> 1964 -> 1923 -> 1944 -> 1945 -> 1938 -> 1937 -> 1939 -> 1935 -> 1931 -> 1934 -> 1673 -> 1675 -> 1674 -> 1837 -> 1671 -> 1828 -> 1825 -> 1817 -> 1815 -> 1634 -> 1814 -> 1813 -> 1632 -> 1631 -> 1742 -> 1741 -> 1740 -> 1739 -> 1591 -> 1689 -> 1585 -> 1584 -> 1688 -> 1579 -> 1679 -> 1677 -> 104 -> 5680 -> 5418 -> 5431 -> 5425 -> 5424 -> 5422 -> 5413 -> 5412 -> 5411 -> 66 -> 5